In [1]:
import os,codecs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.utils.np_utils import to_categorical
from keras import Sequential
from keras.layers import MaxPooling2D
import keras
import cv2
%matplotlib inline

Using TensorFlow backend.


In [3]:
files=os.listdir('../MinorProject/data')

In [4]:
# FUNCTION TO CONVERT TO INT
def get_int(data):
    return int(codecs.encode(data,'hex'),16)

In [5]:
#Reading Data into dictionary
dataset=dict()
path='../MinorProject/data/'                                            #Creating PATH
for file in files:                                                      #READING FILE ONE BY ONE
    path1=path+file                                                     #FILE PATH FOR CURRENT FILE
    print('Reading',path1)
    with open(path1,'rb') as f:                                         #OPENING FILE
        data=f.read()                                                   #READING FILE INTO DATA
        type=get_int(data[0:4])                                         #GETTING MAGIC NUMBER
        length=get_int(data[4:8])                                       #LENGTH OF DATA
        if type==2051:                                                  # IF FILE IS IMAGE SET
            category='images'
            rows=get_int(data[8:12])                                    #GETTING NUMBER OF ROWS
            cols=get_int(data[12:16])                                   #GETTING NUMBER OF COLUMNS
            parsed=np.frombuffer(data,dtype=np.uint8,offset=16)
            parsed=parsed.reshape(length,rows,cols)
        elif type==2049:                                                #IF FILE IS LABEL SET
            category='labels'
            parsed=np.frombuffer(data,dtype=np.uint8,offset=8)
            parsed=parsed.reshape(length)
        if length==60000:
            prefix='train_'
        else:
            prefix='test_'
        dataset[prefix+category]=parsed
print('READING COMPLETE')

Reading ../MinorProject/data/t10k-images.idx3-ubyte
Reading ../MinorProject/data/t10k-labels.idx1-ubyte
Reading ../MinorProject/data/train-images.idx3-ubyte
Reading ../MinorProject/data/train-labels.idx1-ubyte
READING COMPLETE


In [6]:
# CREATING REQUIRED VARIABLES
train_set=dataset['train_images']
train_label=to_categorical(dataset['train_labels'])
test_set=dataset['test_images']
test_label=to_categorical(dataset['test_labels'])
train_set=train_set.reshape(60000,28,28,1)
test_set=test_set.reshape(10000,28,28,1)

In [7]:
#train_set=train_set.reshape(28,28,60000)
#image=train_set[:,:,0]
#plt.imshow(image,cmap='gray')

In [8]:
# CREATING MODEL
model=Sequential()
model.add(Conv2D(32,activation='relu',kernel_size=3,padding='same',data_format="channels_last",input_shape=(28,28,1)))

In [9]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [10]:
model.add(Conv2D(16,activation='relu',padding='same',kernel_size=3))

In [11]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [12]:
model.add(Flatten())

In [13]:
model.add(Dense(128,activation='relu'))

In [14]:
model.add(Dense(64,activation='relu'))

In [15]:
model.add(Dense(10,activation='softmax'))

In [16]:
model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
model.fit(train_set,train_label,validation_split=0.2,epochs=3)

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 16s 337us/step - loss: 0.2650 - accuracy: 0.9339 - val_loss: 0.1085 - val_accuracy: 0.9654
Epoch 2/3
48000/48000 [==============================] - 17s 350us/step - loss: 0.0768 - accuracy: 0.9765 - val_loss: 0.0988 - val_accuracy: 0.9718
Epoch 3/3
48000/48000 [==============================] - 17s 363us/step - loss: 0.0591 - accuracy: 0.9817 - val_loss: 0.0773 - val_accuracy: 0.9780


In [18]:
model.evaluate(test_set,test_label)

10000/10000 [==============================] - 1s 102us/step


[0.06848827003129991, 0.9804999828338623]

In [19]:
model.save('my_model.h5')